# Designing Youtube or Netflix
Let's design a video sharing service like Youtube where users will be able to upload/view/search videos.

Similar Services: Netflix, Vimeo

## 1. Requirements and Goals of the System

#### Functional Requirements:
1. Users should be able to upload videos.
2. Users should have ability to share and view videos.
3. Users should be able to perform searches based on video titles.
4. Service should have starts on videos, e.g likes/dislikes, no. of views, etc.
5. Users should be able to add and view comments on videos


#### Non-Functional Requirements:
- System should be highly reliable, any video uploaded should never be lost.
- System should be highly available.
- Users should have a real time experience while watching videos and shoult not feel any lag.


## 2. Capacity Estimation and Constraints

Assume we have 1 billion total users, 800 million daily active. If on average a user views 5 videos per day, then the total video views per second is:

```
800000000 * 5 / (60 * 60 * 24) ==> ~46000 videos/sec
```

Now, let's assume our upload:view ratio is 1:200 (For every video upload, we have 200 videos viewed)
The number of videos uploaded per second is:

```
46000 / 200 => 230 videos/sec
```


**Storage Estimates:** Lets assume 500 hours of video per minute are uploaeded. If on average, one minute of video needs 50MB of storage(videos need to be stored in multiple formats), then total storage is:

```
500 hours * 60 min * 50 MB => 1500 GB/min OR 25GB/sec
```

We are ignoring video compression and replication, which would change our storage estimates.

**Bandwidth Estimates:** 500 videos/min of uploads, and assuming each video takes a bandwidth of 10 MB/min, we would be getting:

```
500 hours * 60 mins * 10 MB ==> 300 GB/min == 5GB/sec
```

With upload:view ratio being 1:200, we would need 
`200 * 5GB/sec ==> 1TB/s` outgoing bandwidth.